In [ ]:
import os
import pandas as pd

In [ ]:
def merge_csv_files_in_directory(directorio,outputs):
    for directorio, archivo_salida in zip(directorio, outputs):
        dfs = []
        for filename in os.listdir(directorio):
            # Verifica que sean archivos CSV
            if filename.endswith('.csv'):
                filepath = os.path.join(directorio, filename)
                df = pd.read_csv(filepath)
                dfs.append(df)
        merged_df = dfs[0]
        
        for df in dfs[1:]:
            merged_df = pd.merge(merged_df, df, on=['ID_RIO', 'coord_TRAMO'], how='inner')
        
        print(merged_df)
        print("Ahora guardamos en local")
        merged_df.to_csv(archivo_salida, index=False)

In [ ]:
def verificar_permisos_directorio(directory):
    # Verificar si el directorio existe
    if not os.path.exists(directory):
        print(f"El directorio {directory} no existe.")
        return False
    
    # Verificar permisos de lectura
    if not os.access(directory, os.R_OK):
        print(f"No tienes permisos de lectura para el directorio {directory}.")
        return False
    
    # Verificar permisos de escritura
    if not os.access(directory, os.W_OK):
        print(f"No tienes permisos de escritura para el directorio {directory}.")
        return False
    
    # Verificar permisos de ejecución
    if not os.access(directory, os.X_OK):
        print(f"No tienes permisos de ejecución para el directorio {directory}.")
        return False
    
    print(f"Tienes permisos completos para acceder al directorio {directory}.")
    return True

In [ ]:
import os
import pandas as pd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import LineString
import geopandas as gpd
import time

def calcular_distancias(fich_rios, contaminante_sf, fich_output, contaminante_col):
    for idx, rio in fich_rios.iterrows():
        linestring = rio['WKT']
        geometry = loads(linestring)
        
        for part in geometry.geoms:
            #Se reducen el número de tramos a 3
            coords = list(part.coords)
            coordinates_to_check = [coords[0], coords[len(coords)//2] if len(coords) % 2 != 0 else coords[len(coords)//2 - 1], coords[-1]]
            #print("Coordinates to check:",coordinates_to_check)
            #print(f"Número de tramos del rio simplificados: {len(coordinates_to_check)}")
            for coordinate in coordinates_to_check:
                #print("Calculando el sensor más cercano al punto: ",coordinate)
                distances = []
                for idx, row in contaminante_sf.iterrows():
                    sensor = row.geometry.coords[0]  # Obtenemos las coordenadas del sensor
                    distance = np.sqrt((coordinate[0] - sensor[0]) ** 2 + (coordinate[1] - sensor[1]) ** 2)
                    distances.append(distance)

                min_distance = min(distances)
                min_distance_index = distances.index(min_distance)
                closest_sensor = contaminante_sf.iloc[min_distance_index]

                # Crear un DataFrame con los datos
                data = {
                    'ID_RIO': [rio['PFAFRIO']],
                    'coord_TRAMO': [coordinate],
                    f'valor_{contaminante_col}': [closest_sensor['PromedioDe']]
                }
                info_tramo = pd.DataFrame(data)

                # Agregar datos al archivo CSV y Verificar si el archivo existe
                if os.path.exists(fich_output):
                    header = False
                else:
                    header = True

                # Escribir datos en el archivo CSV
                info_tramo.to_csv(fich_output, mode='a', header=header, index=False)

In [ ]:
df_rios_A = pd.read_csv("/Users/silviadelatorre/Desktop/TFG/FICHEROS INPUT/RiosAtlantico.csv")
print(f'Numero de rios Atlantico: {df_rios_A.shape[0]}')
df_rios_M = pd.read_csv("/Users/silviadelatorre/Desktop/TFG/FICHEROS INPUT/RiosMediterraneo.csv")
print(f'Numero de rios Mediterraneo: {df_rios_M.shape[0]}')
# df_rios_A_gdf = gpd.GeoDataFrame(df_rios_A, geometry=gpd.GeoSeries.from_wkt(df_rios_A['WKT']))
# df_fitobentos = pd.read_csv("/Users/silviadelatorre/Desktop/TFG/AGENTES CONTAMINANTES/Fitobentos.csv")
# fitobentos_gdf = gpd.GeoDataFrame(df_fitobentos, geometry=gpd.GeoSeries.from_wkt(df_fitobentos['WKT']))
# output_folder_a = "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO"
# output_path = os.path.join(output_folder_a, "Fitobentos.csv")
# calcular_distancias(df_rios_A_gdf, fitobentos_gdf,output_path,"Fitobentos")

In [ ]:
# UNIR TABLAS EN UNA SOLA
# Directorios de vertientes
vertientes_directorios = ['/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO', '/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/MEDITERRANEO']
output_files = ["/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO/ContaminacionAtlantico.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/MEDITERRANEO/ContaminacionMediterraneo.csv"]

merged_df = merge_csv_files_in_directory(vertientes_directorios,output_files)

In [ ]:
#CONCATENAR FICHEROS DE LAS VERTIENTS
def merge_agents(directory, agent_name):
    dfs = []
    for vertiente in ['ATLANTICO', 'MEDITERRANEO']:
        vertiente_path = os.path.join(directory, vertiente)
        agent_path = os.path.join(vertiente_path, f'{agent_name}.csv')
        if os.path.exists(agent_path):
            df = pd.read_csv(agent_path)
            print(f'Numero de filas {vertiente} en {agent_name} son {df.shape[0]}')
            dfs.append(df)
    merged_df = pd.concat(dfs)
    return merged_df

In [ ]:
# Directorio base donde se encuentran las carpetas ATLANTICO y MEDITERRANEO
base_directory = "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION"

agentes_contaminantes = ['Nitrato', 'Amonio', 'Fosforo', 'Fosfato','Fitobentos','Grado Trofico']
contaminacion_global = pd.DataFrame()

for agente in agentes_contaminantes:
    merged_df = merge_agents(base_directory, agente)
    output_path = os.path.join(base_directory, f'{agente}_merged.csv')
    merged_df.to_csv(output_path, index=False)
    print(f'Numero de filas en {agente} son {merged_df.shape[0]}')
    if contaminacion_global.empty:
        contaminacion_global = merged_df
    else:
        contaminacion_global = pd.merge(contaminacion_global, merged_df, on=['ID_RIO', 'coord_TRAMO'], how='outer')

contaminacion_global.to_csv(os.path.join(base_directory, 'ContaminacionGlobal.csv'), index=False)

In [19]:
#Calcular la suma total de contaminacion por cada rio
contaminacion_global = pd.read_csv("/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ContaminacionGlobal.csv")
contaminacion_global['Contaminación Total'] = contaminacion_global.filter(like='valor_').sum(axis=1)
contaminacion_global.to_csv("/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ContaminacionGlobal.csv", index=False)

In [ ]:
def DibujarGrafica2(Dir,rio,agente):
    '''Grafica de los resultados'''

    # Leer los datos de la tabla
    tabla = os.path.join(Dir, f'{rio}.csv')
    datosHeader = ["Iteracion","Tiempo Infección","Nodos infectados","Ratio infectados","Nodos Totales","pC","pR"]
    datos = pd.read_table(tabla, engine='python', delimiter=' ', header=0, encoding = "ISO-8859-1", names=datosHeader)

    valores_pR = [0.0, 0.1, 0.2, 0.3, 0.4]
    for pR in valores_pR:
        datos_pR = datos[datos['pR'] == pR]

        # Crear una nueva figura para cada valor de pR
        fig = plt.figure()

        # Iterar sobre cada valor de pC y graficar la línea correspondiente
        datos_agrupados = datos_pR.groupby('pC')['Tiempo Infección'].mean().reset_index()

        # Paso 3: Generar la Gráfica
        plt.plot(datos_agrupados['pC'], datos_agrupados['Tiempo Infección'], marker='o', linestyle='-')
        plt.title(f'Mar contaminado de {agente} dado la Probabilidad de Recuperación {pR} en {rio}')
        plt.xlabel('Probabilidad de Contagio (pC)')
        plt.ylabel('Tiempo de Infección Promedio')

        mi_path=os.path.join(Dir, f'pContagio_{rio}_{pR}.png')
    
        plt.legend()
        
        fig.savefig(mi_path)
    plt.close(fig)

In [ ]:
#NUMERO DE SENSORES
sensores = ["/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Amonio.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Fitobentos.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Fosfato.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Fosforo.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Nitratos.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Data/AGENTES CONTAMINANTES/Grafo_Trofico.csv"]
for sensor in sensores:
    df = pd.read_csv(sensor)
    print(f'Numero de sensores en {sensor} son {df.shape[0]}')

In [ ]:
agentes_merged = ["/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Amonio_merged.csv", "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Fitobentos_merged.csv", "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Fosfato_merged.csv", "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Fosforo_merged.csv", "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Grado Trofico_merged.csv", "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/Nitrato_merged.csv"]

for agente in agentes_merged:
    df = pd.read_csv(agente)
    print(f'Numero de filas en {agente} son {df.shape[0]}')